In [1]:
import pandas as pd
order_products_smol = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
aisles = pd.read_csv("../instacart_2017_05_01/aisles.csv")
departments = pd.read_csv("../instacart_2017_05_01/departments.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

In [24]:
orders.head(50)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [3]:
user_orders = orders.merge(order_products_smol,on='order_id', how='inner')

In [4]:
user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [5]:
user_orders = user_orders.merge(products, on='product_id', how='inner')

In [6]:
user_orders = user_orders.merge(aisles, on='aisle_id', how='inner')
user_orders = user_orders.merge(departments, on='department_id', how='inner')
user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1187899,1,train,11,4,8,14.0,196,1,1,Soda,77,7,soft drinks,beverages
1,2757217,67,train,25,0,11,5.0,196,1,1,Soda,77,7,soft drinks,beverages
2,632715,676,train,12,0,13,26.0,196,2,1,Soda,77,7,soft drinks,beverages
3,1167274,760,train,5,4,10,8.0,196,2,1,Soda,77,7,soft drinks,beverages
4,3347074,804,train,16,3,21,5.0,196,4,0,Soda,77,7,soft drinks,beverages


# Unweighed
## Not taking reorders into account

In [7]:
reordercheck = user_orders.groupby(['user_id', 'product_id']).size()
reordercheck[reordercheck > 1]

Series([], dtype: int64)

Guess there are no duplicate items per person

In [8]:
user_orders['order'] = 1

In [9]:
def weigh_orders(order):
    if order['reordered']:
        return 1.3
    else:
        return 1

In [12]:
user_orders['weighted_order'] = user_orders.apply(weigh_orders, axis=1)

In [13]:
user_orders.head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,order,weighted_order
0,1187899,1,train,11,4,8,14.0,196,1,1,Soda,77,7,soft drinks,beverages,1,1.3
1,2757217,67,train,25,0,11,5.0,196,1,1,Soda,77,7,soft drinks,beverages,1,1.3
2,632715,676,train,12,0,13,26.0,196,2,1,Soda,77,7,soft drinks,beverages,1,1.3
3,1167274,760,train,5,4,10,8.0,196,2,1,Soda,77,7,soft drinks,beverages,1,1.3
4,3347074,804,train,16,3,21,5.0,196,4,0,Soda,77,7,soft drinks,beverages,1,1.0
5,692135,865,train,4,5,11,30.0,196,1,0,Soda,77,7,soft drinks,beverages,1,1.0
6,3289231,992,train,22,5,14,30.0,196,6,1,Soda,77,7,soft drinks,beverages,1,1.3
7,1269882,1306,train,7,5,10,6.0,196,9,1,Soda,77,7,soft drinks,beverages,1,1.3
8,1558696,1512,train,4,1,10,27.0,196,1,0,Soda,77,7,soft drinks,beverages,1,1.0
9,2005577,1550,train,14,1,14,30.0,196,6,1,Soda,77,7,soft drinks,beverages,1,1.3


In [18]:
users = user_orders[['user_id', 'product_id', 'weighted_order']]

In [19]:
users.head()

,user_id,product_id,weighted_order
0,1,196,1.3
1,67,196,1.3
2,676,196,1.3
3,760,196,1.3
4,804,196,1.0


In [22]:
users = users.pivot(index='user_id', columns='product_id', values='weighted_order')

MemoryError: 

In [ ]:
users = users.fillna(0)

In [ ]:
users.to_pickle('users.pkl')

In [14]:
import numpy as np
users_p1, users_p2, users_p3, users_p4, users_p5, users_p6, users_p7, users_p8, users_p9, users_p10, users_p11, users_p12 = np.array_split(users, 12)

In [15]:
users_p1.shape

(10935, 39123)

In [16]:
users_p1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10935 entries, 1 to 17100
Columns: 39123 entries, 1 to 49688
dtypes: float64(39123)
memory usage: 3.2 GB


In [17]:
users_p1 = users_p1.fillna(0)
users_p2 = users_p2.fillna(0)
users_p3 = users_p3.fillna(0)


In [18]:
users_p4 = users_p4.fillna(0)
users_p5 = users_p5.fillna(0)
users_p6 = users_p6.fillna(0)
users_p7 = users_p7.fillna(0)
users_p8 = users_p8.fillna(0)
users_p9 = users_p9.fillna(0)
users_p10 = users_p10.fillna(0)

In [19]:
users_p11 = users_p11.fillna(0)
users_p12 = users_p12.fillna(0)

In [20]:
users_p1.head()

product_id,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
split_users = [users_p1, users_p2, users_p3, users_p4, users_p5, users_p6, users_p7, users_p8, users_p9, users_p10, users_p11, users_p12]
for i, split_user in enumerate(split_users):
    filename = "users_p" + str(i + 1) + ".pkl"
    split_user.to_pickle(filename)

OSError: [Errno 22] Invalid argument

In [ ]:
users = pd.concat([users_p1, users_p2, users_p3, users_p4, users_p5, users_p6, users_p7, users_p8, users_p9, users_p10, users_p11, users_p12], axis=0)

In [ ]:
users.head()

In [ ]:
users.shape

In [ ]:
import numpy as np
users_matrix = users.values()
users_mean = np.mean(users_matrix, axis = 1)
users_demeaned = users_matrix - users_mean.reshape(-1, 1)

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(users_demeaned, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_purchases = np.dot(np.dot(U, sigma), Vt) + users_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_purchases, columns = users.columns)